###### Coursera  | Andrew Ng's Deep Learning Class | Course2. Improving Deep Neural Networks_Hyperparameter tuning, Regularization and Optimization | Week1. Practical aspects of Neural networks

이번주에는 하이퍼 파라미터 튜닝, 데이터를 어떻게 나눌 것인지, 최적화 알고리즘의 실행 속도를 증가시키는 방법, 정규화, 모형이 정상적으로 동작하는지 확인하는 방법 등 신경망 모형을 좀 더 잘 동작하게 만드는 실질적인 방법들에 대해 알아본다. 

# [1]. Setting up your Machine Learning application

### [1-1]. Train/dev/test sets

딥러닝 모형을 잘 동작하게 하려면 레이어의 수, 각 히든 레이어에 유닛을 몇개식 둘 것인지, learning rate은 얼마로 할 것인지, 활성함수로는 어떤 것을 사용할 것인지 등 결정해야 하는 요소들이 매우 많다. 새로운 모형에 대해 이런 요소들을 바로 잘 결정하는 건 거의 불가능한 일이라서 어떤 아이디어대로 이런 요소들을 정하고, 이것들을 구현하고, 데이터에 적용 해보고, 아이디어를 수정하여 다시 구현하고 적용하는 과정을 반복하면서 더 나은 방향을 찾는 방식이 일반적이다. 또한 특정 도메인에서 잘 적용되던 노하우가 다른 도메인에서는 크게 효과가 없는 경우가 많아서 이런 반복적인 과정이 필수적이다. (예를들어 NLP에서 잘 적용되던 방식이 비전 문제에는 효과가 없을 수도 있다.)

따라서 이런 반복 과정(아이디어 ⇒ 구현 ⇒ 실험 ⇒)을 얼마나 빠르고 효과적으로 많이 수행할 수 있느냐가 좋은 모형을 만드는데 매우 중요한 요소일수밖에 없다.

![Local image](./images/C2W1L01_01.PNG)

확보한 데이터를 일반적으로 **훈련데이터**(Training set), **개발데이터**(Hold-out cross validation set(=Development set)), **테스트데이터**(Test set)로 나눈다. 우선 훈련데이터로 모형 파라미터($W$, $b$..)를 학습시키고, 개발데이터로 (하이퍼 파라미터를 달리하는) 여러 모형중 성능이 나은 것을 찾고, 테스트데이터로 최종 모형에 대한 객관적인(unbiased) 모형의 성능을 측정한다.

과거의 전통적인 머신러닝에서는 전체 데이터를 6:2:2비율로 훈련데이터/개발데이터/테스트데이터로 나눠 사용하거나, 7:3 비율로 훈련데이터/개발데이터로 나누는 것이 일반적이었다. 그런데 이런 분할 비율은 전체 데이터가 몇백만건에서 몇만건 정도일 경우에는 합리적인 선택었다. 하지만 요즘과 같이 몇백만건 이상의 데이터를 다루는 것이 일반적인 상황에서는 예기가 달라진다. 사실 개발데이터는 여러 모형 중 어느것이 더 나은지만 빠르게 선택할 수 있는 정도면 충분하고, 모형의 객관적 성능 측정 또한 그리 많은 데이터가 필요하지 않다. 몇백만건의 데이터가 있을 경우 개발데이터와 테스트데이터에 몇만건 정도면 충분하며 이는 전체 데이터의 1%정도에 불과하다. 데이터가 더 많을 경우 전체 데이터의 0.5%이하 데이터 만으로도 충분히 개발데이터와 테스트데이터로 활용할 수 있다.

![Local image](./images/C2W1L01_02.PNG)

데이터를 구성하는데 있어 또 한가지 중요한 것은 훈련데이터와 개발/테스트 데이터의 분포가 같아야 한다는 점이다. 예를들어 고양이 그림을 구분하는 모형을 만든다고 할 때 훈련데이터는 사용자가 웹페이지에 업로드한 사진들을 사용하고, 개발/테스트데이터는 사용자가 모바일앱으로 올린 데이터를 사용하는 경우 두 데이터는 해상도/크기/구도 등 특성이 다를 것이기 때문에 훈련데이터로 학습한 모형을 개발/테이터가 잘 평가할 수 없을 것이다. 

그리고 모형 성능에 대한 객관적 지표(Unbiased estimate for the model performance)가 필요 없는 경우 테스트 데이터를 별도로 두지 않아도 된다. 이 경우 데이터는 훈련데이터와 개발데이터로 나뉠텐데, 이때의 개발데이터를 테스트데이터로 부르는 경우도 있으나 바람직한 명칭은 아니다. (하이퍼파라미터 튜닝과 모형 선택을 위해 사용되는 데이터를 개발데이터로 부르는 것을 권장한다.)

![Local image](./images/C2W1L01_03.PNG)

### [1-2]. Bias/Variance

Bias/Variance는 얼핏 보기에는 쉬운 것 같지만 사실 마스터하기는 간단치 않은 개념이며, 훌륭한 머신러닝 실무자들은 대부분 Bias/Variance를 아주 잘 이해하고 있다. 그만큼 정확한 이해가 중요한 개념이다.

잘 알다시피 머신러닝에 있어서 Bias/Variance를 다루는데 있어서 이 둘 사이에 절충점을 찾는 것(trade-off)이 중요하다. 그런데 딥러닝에서는 Bias를 해결하려는 것과 동시에 Variance도 해결하려 하는데 이 둘 사이의 trade-off에 대해서는 잘 논하지 않는다. 이 말이 무슨 뜻인지 차차 알아보도록 한다.

---

아래와 같이 O와 X로 구성된 이항 데이터가 있고 이들을 O와 X로 나누려는 어떤 선을 찾는다고 해 보자.

우선 첫번째 그림과 같은 직선을 사용하여 직선 위 점은 X로, 직선 아래 점들은 O라고 예측할 경우 상당히 많은 오-예측이 발생한다. 이런 경우를 'high bias' 혹은 under-fitting이라 한다. 반면 극단적으로 세번째 그림과 같이 주어진 데이터에 아주 잘 맞는 아주 복잡한 곡선을 사용할 경우의 상황을 'high variance' 혹은 over-fitting이라 한다. 이 두 경우의 절충형으로 두번째 그림과 같이 적당히 굴곡진 곡선을 사용할 경우 약간의 오차가 있지만 많은 경우 잘 들어맞는 예측을 할 수 있게 된다. 

데이터의 설명변수($x$, feature)가 둘뿐인 경우 이렇게 2차원상에 decision boundary를 표현할 수 있지만, 차원이 많아질 경우 시각화하는 것이 어려워진다.(물론 다차원 데이터를 시각화하기 위한 기법들이 있지만 2차원상에 표현하는 순간 많은 정보를 잃게 된다.)

![Local image](./images/C2W1L02_01.PNG)

(앞서 decision boundary를 시각화 방법외에) Bias/Variance 문제를 점검하기 위해 훈련데이터와 개발데이터에 대한 예측 에러를 확인하는 방법도 있다. 아래 그림과 같이 고양이 그림과 고양이가 아닌 그림들로 구성된 데이터가 있을 때 고양이 그림을 판별하는 모형을 만든다고 해 보자. 4가지 모형을 만들었고 훈련데이터와 개발데이터에 대한 에러는 아래 슬라이드와 같다. 

우선 첫번째 경우는 훈련데이터에 대해서는 아주 좋은 성능을 보이지만 개발데이터에 대해서는 성능이 상대적으로 좋지 않다. 이는 훈련데이터에 과적합(over-fitted)된 상태이고 잘 일반화(generalize)되지 않는 모형이다. 이런 경우 개발데이터 에러가 훈련 데이터 에러와 큰 차이를 보이는 상태를 표현하여 'high variance'라 한다.

두번째 경우는 훈련데이터에 대해서 성능이 좋지 않고, 개발데이터에 대해서는 훈련데이터에 비해 성능 차이가 크지 않지만 이 또한 성능이 좋지 않다. 이 모형은 훈련데이터에 부적합(underfitting)하며 개발데이터에 일반화(generalize)는 잘 된다고 할 수 있다. 이런 경우 훈련 데이터에러가 (기대보다) 큰 상황을 표현하여 'high bias'라 한다.

세번째 경우 훈련데이터 에러가 높고, 개발데이터 에러는 이와 큰 차이로 더 높은 상황이다. 이 경우 두번째 경우와 마찬가지로 훈련데이터 에러가 크므로 'high bias'이고, 첫번째 경우와 마찬가지로 훈련데이터 에러와 테스트 데이터 에러가 큰 상황이므로 'high variance'라 한다. 이는 최악의 상황이다.

네번재 경우 훈련데이터 에러도 낮고, 이와 작은 차이로 개발데이터 에러가 낮으므로 'low bias', 'low variance'라 한다. 이는 아주 바람직한 상황이라 할 수 있다.

이런 논의에서 훈련데이터 에러가 높은지 낮은지에 대한 판단 기준은 어떻게 정한 것일까? 일반적으로 사람이 이런 그림을 판별할 경우 거의 에러가 없을 것이므로 훈련데이터에 대한 기대 에러 또한 매우 낮을 것이고 이 수준($0%$에러)를 optimal error 혹은 bayes error라 하고 이 기대 수준과 훈련 데이터 에러를 비교하여 'bias'가 큰지 작은지를 판단한다.

결국 기대수준(optimal error)과 훈련데이터 에러를 비교하여 high bias 문제가 있는지를 판단하며, 훈련데이터 에러와 개발데이터 에러를 비교하여 high variance 문제가 있는지를 판단한다.

마지막으로 이런 논의는 bayes error가 꽤 낮고, 훈련데이터와 개발데이터가 동일한 분포에서 나왔다는 것을 가정하고 있다. 이런 가정이 성립하지 않을 경우 좀더 복잡한 고려가 필요하다.

![Local image](./images/C2W1L02_02.PNG)

.

![Local image](./images/C2W1L02_03.PNG)

.

.

### [1-3]. Basic "recipe" for machine learning

앞서 '훈련데이터 에러'와 '개발데이터 에러'를 통해 현재 모형의 bias/variance 문제를 진단할 수 있다고 했었다. 이런 진단을 통해 모형의 high bias 혹은 high variance 문제(혹은 둘다)가 발견될 경우 다양한 방법(recipe)을 이용해 모형을 변형해 볼 수 있고, 다시 훈련데이터 에러와 개발데이터 에러를 확인하여 상황이 개선되는지 아닌지를 판단할 수 있다.

bias/variance 문제에 대한 진단과 처방의 일반적인 과정은 다음과 같다. 우선 최초 모형을 학습시키고 모형에 high bias 문제가 있는지 확인한다.(즉 훈련데이터에 잘 적합하는지 확인한다.)

만약 **high bias 문제가 있다면**, <u>문제가 해결될 때까지 반복적으로</u> 아래와 같은 방법들을 시도한다. 
1. 히든 유닛의 수와 히든 레이어 수를 조절하여 보다 큰 네트워크를 이용해 보거나,
2. 최적화를 좀 더 오래 진행해보거나 좀더 나은 최적화 기법을 이용해 보거나,
3. 다른 형태의 네트워크 구조를 사용해 본다.       
(그런데 첫번째 방법은 항상 bias 문제에 도움이 되며, 두번째 문제는 종종 도움이 되지만 문제를 악화시키지는 않으며, 세번째 방법은 도움이 될때도 있지만 문제를 악화시키기도 한다.)   

보통의 경우 큰 네트워크(많은 히든 유닛과 많은 히든 레이어)를 사용하면 훈련데이터에는 점점 더 잘 적합하게 되고, 훈련데이터 에러는 거의 bayes error에 근접하게 된다.

위 접근방법들을 통해 bias 문제가 해결되면(=훈련데이터에 잘 적합하게 되면), 다음으로 variance 문제가 있는지 확인한다.(즉 훈련데이터만큼이나 개발데이터에도 잘 접합하는지 확인한다.)

만약 **high variance 문제가 있다면**, 아래 방법들을 시도해서 variance 문제가 개선되는 방법을 찾고, 다시 bias 문제가 발생했는지 확인하고, 문제를 개선하는 과정을 반복한다.
1. 더 많은 데이터를 확보하고 훈련에 투입해본다.
2. 정규화(regularization) 기법을 사용해본다.
3. 다른 형태의 네트워크 구조를 사용해 본다.     
(데이터 확보가 가능하다면 첫번째 방법이 variance 문제의 가장 좋은 해결책이고, 세번째 방법은 종종 문제를 악화시킨다.)

이렇게 high bias 문제이냐 혹은 high variance 이냐에 따라 문제에 대한 접근법이 달라진다. 예를들어 더 많은 훈련 데이터를 사용하는 것이 high bias 문제를 해결하는데 큰 도움이 되지 않는다. 따라서 모형 성능 개선을 위해서는 문제가 무엇인지 잘 진단하는 것이 중요하다.

---

알다시피 (딥러닝 이전의) 머신러닝에서는 bias를 높여서 variance를 낮추거나 혹은 그 반대의 방식과 같이 한쪽을 희생하여 다른 하나를 얻는 trade-off 식의 접근 방법을 사용했었다. 이럴수밖에 없었던 이유는 당시 다루는 데이터의 양이나 기법으로는 bias를 높여서 variance를 낮추거나 variance를 높여서 bias를 낮추는 것밖에 할 수 없었기 때문이다.

그런데 딥러닝/빅데이터 시대가 되면서 큰 네트워크를 적용하거나 더 많은 데이터를 투입하는 것이 가능해지면서 bias나 variance 한쪽을 크게 훼손하지 않으면서도 두 문제를 적절히 해결할 수 있게 되었다. 

예를들어 정규화(Regularization)을 잘 적용한 경우 더 큰 네트워크를 사용하더라도 variance문제를 야기하지 않으면서 bias를 낮출 수 있게 된다. 다만 네트워크가 커지면 더 많은 컴퓨팅 성능이 필요하다는 점이 부담이라 할 수 있다.

![Local image](./images/C2W1L02b_02.PNG)

.

# [2]. Regularizing your neural network

모형에 과적합 문제(overfitting, high variance)가 있을 때 <u>더 많은 데이터를 학습에 투입하는 것</u>이 가장 효과적이다. 하지만 때로는 추가적인 데이터를 확보하는 것이 비용 등 현실적인 이유로 어려울수 있다. 그래서 가장 먼저 손쉽게 high variance 문제를 해결하기 위해 시도해볼 수 있는 방법이 바로 <u>정규화(regularization)</u>이다.

### [2-1]. Regularization

우선 로지스틱 회귀모형에 정규화가 어떻게 동작하는지 알아보기로 한다.

기본적으로 로지스틱 회귀모형에서는 비용함수 $J(w, b)=\frac{1}{m}\sum_{i=1}^m \mathcal{L}(\hat{y}^{(i)}, y^{(i)})$를 최소화하는 $\mathbf{w}$와 $b$를 찾으려 한다. 여기에 정규화를 적용하려면 비용함수를 아래와 같이 바꾼다.

$$J(w, b)=\frac{1}{m}\sum_{i=1}^m \mathcal{L}(\hat{y}^{(i)}, y^{(i)}) + \frac{\lambda}{2m}\Vert w \Vert_2^2$$

위 식은 두개의 항의 합인데, 첫 항인 $\mathcal{L}(\hat{y}^{(i)}, y^{(i)})$ 값을 작게하는 것은 훈련데이터에 대한 적합도를 높이려는 의도(bias를 낮추는)를 갖고, 두번째 항인 $\frac{\lambda}{2m}\Vert w \Vert_2^2$는 $\mathbb{w}$ 값들을 작게 하려는 의도(variance를 낮추는)를 갖는다.

정규화를 적용하는 방식이 여러가지가 있는데 위에서 사용한 $\Vert w \Vert_2^2$는 $L_2$ 정규화(혹은 lasso)라 하며, 가장 일반적으로 사용되고, 파라미터 $w$들의 제곱합을 의미한다. 반면 $||w||_1$과 같이 표현하는 것은 $L_1$ 정규화(혹은 Ridge)라 하며 파라미터 $w$들의 절대값의 합을 의미한다. 참고로 $L_1$ 정규화는 많은 $w$값들이 0이 되게하는(sparse)하여 모형을 압축하는 효과를 갖는다.(0 파라미터가 많아질수록 cache하는 모형의 용량이 작아진다. 그런데 모형이 아주 아주 크고 강한 정규화를 사용하는 극단적인 상황이 아니라면 그 효과가 크지는 않다.)

$w$들 뿐만 아니라 $b$의 크기 또한 억제해야 하기 위해 위 식에 $\frac{\lambda}{2m}b^2$를 더할수도 있지만 사실 결과에 큰 차이를 만들지 않기 때문에 $\frac{\lambda}{2m}b^2$는 생략해도 무방하다.

위 식에서 $\lambda$는 정규화의 정도를 표현하는 상수(regularization parameter)이고, 이 값을 크게 하면 그만큼 variance가 커지는 것을 억제하겠다는 것이다. 적절한 $\lambda$값을 찾는 것이 중요한데, 개발데이터(dev set)이나 테스트데이터(test set, cross-validation set)을 이용해 여러가지 $\lambda$값을 테스트 하여 가장 성능이 좋은 값을 선택한다.

![Local image](./images/C2W1L03_01.PNG)

이제 신경망에서의 정규화(regularization)에 대해 알아보자.

신경망의 비용함수(cost function)는 $J(W^{[1]}, b^{[1]}, \cdots, W^{[L]}, b^{[L]}) = \frac{1}{m}\sum_{i=1}^m \mathcal{L}(\hat{y}^{(i)}, y^{(i)})$이고 여기에 정규화를 위한 항 $\frac{\lambda}{2m}\sum_{l=1}^L \Vert W^{[l]}\Vert^2_F$를 추가하면 아래와 같아진다.(이하 논의에서 bias항 $b$에 관한 것은 생략한다.)

$$J(W^{[1]}, b^{[1]}, \cdots, W^{[L]}, b^{[L]}) = \frac{1}{m}\sum_{i=1}^m \mathcal{L}(\hat{y}^{(i)}, y^{(i)}) + \frac{\lambda}{2m}\sum_{l=1}^L \Vert W^{[l]} \Vert^2_F$$

$W^{[l]}$은 $l-1$번째 레이어와 $l$번째 레이어 사이의 weight 행렬로서 shape은 ($n^{[l]}$, $n^{[l-1]}$)이다. 레이어가 총 $L$개 있으므로 이런 행렬이 $L$개 있고 이들의 크기가 지나치게 커지지 않게 하는 것이 정규화의 목적이다. 앞서 로지스틱 회귀에서 L1 정규화와 L2 정규화가 있었는데 여기에서는 L2와 비슷한 방식을 사용하여, $L$개 weight 행렬의 원소를 제곱하고 모두 더한 값을 $\Vert W^{[l]} \Vert^2_F$라 표현하며 이 값에 $\frac{\lambda}{2m}$을 곱한 $\frac{\lambda}{2m}\sum_{l=1}^L \Vert W^{[l]} \Vert^2_F$을 정규화 항으로 사용한다.

선형대수에서는 $m$행 $n$열 행렬 $A$가 있을 때 행렬의 각 원소를 제곱하여 더하고 제곱근을 취한 값($\sqrt{\sum_{i=1}^m \sum_{j=1}^n \vert a_{ij} \vert^2}$)을 프로비니우스 놈(Frobenius norm)이라하며 $\Vert A \Vert_F$라고 표기한다. ($\Vert A \Vert$뒤에 밑첨자 $_F$를 붙인 이유이다.)

따라서 $\Vert W^{[l]} \Vert^2_F$은 프로비니우스 놈을 제곱한 값이며, $L$개 weight 행렬의 원소들을 제곱하고 모두 더한 값이다.

비용함수를 이용해 결국 하려는 것은 비용함수 값이 최소가 되도록 $W$들을 수정해나가는 것이며, 이를 위해서는 $\frac{\partial J}{\partial W^{[l]}}$를 구해야 한다.(이전 강의에서와 같이 이 편미분값을 $dW^{[l]}$로 표기할 것이다.) $\frac{\partial J}{\partial W^{[l]}}$는 $\frac{1}{m}\sum_{i=1}^m \mathcal{L}(\hat{y}^{(i)}, y^{(i)})$를 $W^{[l]}$에 관해 편미분한 값과 $\frac{\lambda}{2m}\sum_{l=1}^L \Vert W^{[l]} \Vert^2_F$를 $W^{[l]}$에 관해 편미분한 값의 합이며 첫번째 편미분항은 일반적인 역전파과정에서 구해지므로 'from backprop'이라 부를 것이다. 두번째 편미분항은 $\frac{\lambda}{m}W^{[l]}$로서, 단지 $W^{[l]}$에 $\frac{\lambda}{m}$을 곱한 값이다.

$W^{[l]}$에 대한 업데이트 방식은 아래와 같은데
$$\begin{align}
W^{[l]} &:= W^{[l]} - \alpha dW^{[l]} \\
&= W^{[l]} - \alpha \left[ (\text{from backprop}) + \frac{\lambda}{m}W^{[l]} \right]\\
&= W^{[l]} - \frac{\alpha \lambda}{m} W^{[l]} - \alpha (\text{from backprop}) \\
&= (1-\frac{\alpha \lambda}{m}) W^{[l]} - \alpha (\text{from backprop}) \\
\end{align}$$
이는 정규화 하지 않을 때와 비교하면 $W^{[l]}$에 $(1-\frac{\alpha \lambda}{m})$을 곱하는 차이밖에 없고, $(1-\frac{\alpha \lambda}{m})$ 값은 1보다 작기 때문에 매 업데이트에서 $W^{[l]}$를 조금식 작게 유지하는 효과를 가져오게 된다. 이런 이유 때문에 L2 정규화를 **weight decay**라 부르기도 한다.

![Local image](./images/C2W1L03_02.PNG)

### [2-2]. Why regularization reduces overfitting

.

![Local image](./images/C2W1L03b_01.PNG)

.

![Local image](./images/C2W1L03b_02.PNG)

.

.

### [2-3] Dropout regularization

'L2 정규화'와 더불어 매우 효과적인 정규화 방법이 바로 '**drop-out**' 이다. drop-out은 말 그대로 노드들을 없애겠다는 것인데, 노드들을 유지할 확률(keep probability)을 정하고 무작위로 이 확률만큼만 노드들을 남기고 나머지는 없애버린다. 이렇게 일부 노드가 사라지면 아래 우측 그림과 같이 네트워크는 한결 단순해지고 모형 복잡도가 감소하게 된다.

![Local image](./images/C2W1L04_01b.PNG)

drop-out을 구현하는 다양한 방법이 있는데 여기에서는 'Inverted dropout'이라는 방식에 대해 알아본다.

예를들어 4번째 히든 레이어에서의 drop-out을 생각해 보자. mini-batch 방식(mini-batch size = $k$)으로 구현한 경우 3번째 히든 레이어의 activation은 shape이 ($n^{[3]}$, $k$) 인 행렬 $a^{[3]}$이고, 이 행렬의 각 열(샘플) 마다 4번째 히든 레이어의 keep/drop이 달라진다. 

4번째 히든 레이어의 어떤 유닛을 남길지 결정하기 위해 ($n^{[3]}$, $k$)개의 난수를 생성하고, 그 값이 'keep probability' 보다 높을 경우 그 노드는 남기고 그렇지 않을 경우 없앤다. 실제로는 유닛이 사라지는 효과를 얻기 위해 해당하는 노드가 사라지는 위치의 $a^{[3]}$값에 0을 곱한다. 

그런데 유닛이 사라지면 그만큼 $a^{[3]}$의 기댓값 또한 줄어들게 되고 이는 $z^{[4]}$에도 영향을 주게 된다. 이런 부작용을 줄이기 위해 $a^{[3]}$를 'keep probability'로 나눠서 $a^{[3]}$의 기댓값이 변하지 않게 해준다. 이렇게 나눠주는 방식을 'Inverted dropout' 이라 한다.

In [16]:
import numpy as np
n3 = 3
k = 7
keep_prob = 0.8

a3 = np.random.randn(n3, k)
d3 = np.random.randn(*a3.shape) < keep_prob
a3 = np.multiply(a3, d3)
a3 /= keep_prob
a3

array([[ 1.28668085,  1.77072853, -0.        , -1.87547732,  0.02684609,
        -1.05027343, -0.        ],
       [-2.25355859, -2.36193646,  1.85009466,  0.64070639, -0.46415974,
        -1.51777603,  0.2953842 ],
       [-2.80100814,  0.6305165 ,  0.78704658, -0.74735055,  2.54235128,
         0.83618093,  2.51979217]])

![Local image](./images/C2W1L04_02.PNG)

drop-out을 이용해 모형을 학습시키고나서 개발/테스트 데이터에 대한 예측을 해 볼텐데 이때에는 drop-out을 사용하지 않는다. 물론 예측시에도 drop-out을 적용하고 랜덤한 결과들을 종합(평균)하여 사용할수도 있지만 이 결과가 drop-out을 사용하지 않은 결과가 거의 비슷하다.

![Local image](./images/C2W1L04_03.PNG)

### [2-4] Understanding dropout

drop-out이 정규화에 효과적인 이유가 뭔지 알아본다. 이전 슬라이드 [2-3]에서는 drop-out이 적용되면 iteration마다 상대적으로 작은 네트워크를 사용하는 것과 같고 이것이 정규화 효과를 갖는다고 설명했었다.

다른 관점에서 살펴보면, 아래 슬라이드 좌측의 네트워크와 같이 보라색 노드가 몇개의 이전 노드에서 입력을 받는다고 할 때, 랜덤하게 입력 노드가 사라지기 때문에 어느 특정 노드 입력에만 큰 가중치(weight, $w$)를 주기 어려울 것이다. 즉 drop-out이 적용되면 weight이 레이어의 여러 노드에 분산(spread-out)되는 효과가 나타나게 된다. weight이 분산된다는 것은 weight의 제곱합이 줄어드는 것과 같고 이는 L2 정규화와 같은 효과이다.

실제로 drop-out이 adaptive한 형태의 L2 정규화라는 것을 보일 수 있다. 여기서 adaptive하다는 것은 weight 마다 activation의 크기에 따라 유동적으로 정규화의 강도(L2 penalty)를 조절한다는 것이다. (절대)값이 큰 activation을 drop한 경우 큰 L2 penalty를 적용했다고 볼 수 있다.

---
또 한가지 팁은 drop-out을 적용할 때 레이어마다 다른 keep probability를 사용할수도 있다는 것이다. over-fitting 우려가 큰 레이어에는 작은 keep probability를 사용하고 상대적으로 단순한 레이어에는 1에 가까운 keep probability값을 사용하는 것이다. 이때 작은 keep probability값을 사용한다는 것은 L2 정규화에서 큰 값의 정규화 상수 $\lambda$를 사용하는 것과 유사한 효과를 갖는다.

아래 슬라이드 우측과 같은 네트워크에서는 2번 레이어에는 7x7의 weight 행렬이 사용되므로 복잡도가 상대적으로 크고 이 부분에서 over-fitting이 발생할 가능성이 크다. 따라서  2번 레이어의 keep probability를 작은 값으로 설정한다. 반면 5번 레이어의 경우 유닛이 하나뿐이므로 keep probability로 1.0을 사용한다. 

그리고 입력 레이어에 drop-out을 적용하는 것이 가능하기는 하다. 하지만 실무에서는 입력 레이어에는 drop-out을 잘 적용하지 않는다. 즉 0번째 레이어의 keep probability로 1 혹은 1에 아까운 값을 사용한다.

이렇게 레이어마다 다른 keep probability를 사용할 때 구현상의 단점은 레이어 마다 적절한 keep probability를 정해야 한다는 점이다. 혹은 drop-out을 적용할 레이어들을 결정하고 이들 레이어에 동일한 keep probability값을 사용하는 방법도 있다.

---
컴퓨터 비전 분야에서는 입력데이터의 차원이 이미지의 픽셀수만큼 크기 때문에 거의 항상 훈련 데이터 부족하고 이때문에 거의 항상 과적합 문제가 발생한다. 따라서 컴퓨터 비전에서는 drop-out을 매우 자주 사용한다. 그런데 drop-out이 정규화를 위한 방법이기 때문에 현재 모형에 과적합 문제가 없다면 굳이 drop-out을 사용할 필요가 없다.

---
drop-out의 또 다른 단점은 랜덤하게 노드들이 사라지기 때문에 비용함수 $J$가 잘 정의되지 않는다. 이 때문에 cost-iteration 그래프로 iteration마다 cost가 단조감소하는지 확인하는 것이 어렵다. 실무에서는 gradient descent가 잘 동작하고 있는지 확인해야 할 때는 keep probability를 1로 설정하여 drop-out을 끄고 cost-iteration 그래프를 확인하고 다시 drop-out을 켜는 방식을 쓰기도 한다.

![Local image](./images/C2W1L04b_01.PNG)

### [2-5]. Other regularization methods

L2 정규화, drop-out 외에 과적합을 피하기 위한 다른 방법들도 있다.

아래 슬라이드와 같이 고양이 그림을 판별하는 모형을 만들었는데 과적합 문제가 발생했다면 가장 쉬운 접근법은 데이터를 더 투입하는 것이다. 그런데 고양이 그림을 더 획득하는 데 비용이 많이 든다는 등 현실적인 문제가 있다면 이미 확보한 데이터를 변형하여 투입하는 것도 좋은 방법일 수 있다. 예를들어 고양이 그림을 좌우로 대칭변환한 그림이나 랜덤하게 일부 잘라낸 그림등을 사용하는 것이다. 이런 접근법을 **data augmentation**이라 한다.

문자 인식 문제에 있어서도 문자들을 회전하거나 찌그러뜨리는 등의 변형을 준 이미지를 훈련 데이터에 추가하는 방법을 취할 수 있다.

![Local image](./images/C2W1L05_01.PNG)

또 다른 접근법으로 **early stopping**이란 것도 있다. 학습 진행도중 멈춘다는 것인데,  아래 슬라이드와 같이 훈련데이터와 개발데이터에 대한 cost-iteration 그래프를 그려보면 일정 iteration 이후에는 개발데이터에 대한 에러가 증가하여 과적합이 진행된다는 것을 알 수 있다. 따라서 학습을 계속 진행하지 않고 개발데이터에 대한 에러가 증가하는 시점에서 학습을 중단하는 것이다. 

학습을 시작하기 전에 weight들을 0에 가까운 랜덤한 값들로 초기화 하기 때문에 이렇게 일찍 학습을 중단시키면 많은 weight 값들이 0에 가까운 값인 상태로 멈추게 된다. 이 때문에 모형의 과적합 가능성이 줄어드는 효과가 나타난다.

---
그런데 early stopping에는 한가지 단점이 있다.

머신러닝 과정은 몇가지 단계로 나뉘는데,       
- 첫번째로 비용함수 $J$를 최소화 하는 점을 찾는 것(최적화, optimize)이다. 이를 위한 방법으로 Gradient descent, Momentum, RMSProp, Adam 등이 있다. 이런 방법들 중 하나로 비용함수 $J$를 최적화 한 후,          
- 두번째 단계로 과적합 문제를 해결해야한다. 이를 위한 방법으로 정규화(L2, L1, ...), drop-out, 더 많은 데이터 확보 또는 data augmentation, early stopping 등이 있다.

그런데 최적화만을 위해서도 결정해야할 요소들(hyper-parameters)이 많기 때문에 최적화 단계에서는 오직 $J$를 최소화 하는 $W$와 $b$를 찾는것에만 집중하는 것이 좋다. 이 단계를 마치고나서 과적합 해소와 같은 다음 단계의 문제를 해결하는 것이 바람직히다. 이렇게 한번에 하나의 문제에만 집중하는 방식을 직교화(orthogonalization)라 한다. 

그런데 early stopping은 위 두 단계를 섞이게 만든다. 즉 최적화를 하면서 동시에 과적합 문제 또한 해결하려 한다. 이렇게 되면 최적화와 과적합 문제를 풀어내는 것이 복잡해지고 둘다 망치게될수도 있다. 

오히려 L2 정규화를 적용하고 여러 정규화 상수 $\lambda$값을 테스트 하여 가장 좋은 성능을 보이는 $\lambda$값을 선택하는 방법이 더 효과적일 수 있다. 이렇게 하면 최적화 단계는 최대한 $J$를 줄이는 것에만 집중하게 되고, 과적합 측면에서는 여러 $\lambda$값을 테스트 하기만 하면 된다. 다만 이 접근법은 여러 $\lambda$값을 테스트 해야하므로 계산량이 많아지는 단점이 있다.

이런 측면에서는 early stopping은 gradient descent 과정을 한번만 진행하면 된다는 장점이 있다. 즉 일정 iteration 횟수마다의 $W$값을 이용해 개발데이터 에러를 체크하고 개발 데이터 에러가 증가하기 이전의 iteration일때 $W$값을 사용하면 되는 것이다.

![Local image](./images/C2W1L05_02.PNG)

# [3]. Setting up your optimization problem

최적화에 도움이 되는 몇가지 팁에 대해 알아본다. 

### [3-1]. Normalizing inputs

최적화의 속도를 높이는 방법중 하나가 입력데이터 표준화(normalize)하는 것인데, 입력데이터 행렬 $X \in \mathbb{R}^{p \times m}$에서 각 행의 평균을 빼고 각 행의 분산으로 나누는 것이다. 
$$X_{j, ~ \text{normalized}} = \frac{X_j - \mu_j}{\sigma^2_j}, \qquad j \in \{1, 2, \cdots, p\}$$

평균 $\mu_j$는 $\frac{1}{m}\sum_{i=1}^m x_j^{(i)}$와 같이, 분산 $\sigma_j^2$는 $\frac{1}{m} \sum_{i=1}^m {(x^{(i)}_j)}^2$와 같이 계산하는데, 훈련데이터에 대해 구한 $\mu$와 $\sigma^2$를 개발/테스트 데이터를 표준화하는데에도 사용한다. 이는 훈련/개발/테스트 데이터의 변환 기준을 동일하게 하기 위함이다.

![Local image](./images/C2W1L06_01.PNG)

그렇다면 왜 입력데이터를 표준화 하는 것일까.

입력 변수들의 단위가 1~1000, 0~1과 같이 매우 다르고 표준화 하지 않은 경우 비용함수는 아래 슬라이드의 좌측 그림과 같이 매우 길쭉한 형태가 될 것이다. 이 경우 gradient descent로 $J$의 극소점을 찾기 위해서는 대단히 많은 반복이 필요하게 된다. 반면 표준화한 경우 비용함수는 아래 우측과 같은 그림이 되는데, 이 경우 표준화 하지 않은 경우에 비해 훨신 적은 반복만으로 극소점에 도달하게 된다. 즉 입력 변수값의 단위가 비슷하면 비용함수의 형태가 보다 둥글게 되고 최적화 또한 더 빠르게 진행된다. 

따라서 입력데이터의 변수간 단위가 크게 다를 경우 표준화하는 것이 최적화 속도를 높이는데 도움이 될 것이다. 하지만 단위 차이가 크지 않을 경우 큰 차이는 없을 것이다.

![Local image](./images/C2W1L06_02.PNG)

.

### [3-2]. Vanishing/exploding gradients

신경망을 학습시킬 때 특히 매우 깊은 모형을 학습시킬 때 겪을 수 있는 문제가 vanishing gradient 혹은 exploding gradient 이다.


![Local image](./images/C2W1L07_01.PNG)

.

### [3-3]. Weight initialization for deep networks

.

![Local image](./images/C2W1L07_02.PNG)

.

### [3-4]. Numerical approximation of gradients

.

![Local image](./images/C2W1L08_01.PNG)

.

![Local image](./images/C2W1L08_02.PNG)

.

### [3-5]. Gradient Checking

.

![Local image](./images/C2W1L09_01.PNG)

.

![Local image](./images/C2W1L09_02.PNG)

.

### [3-6]. Gradient Checking implementation notes

.

![Local image](./images/C2W1L09b_01.PNG)

.

.

.

.

만약 확보한 데이터가 찌그러지고 흐린 이미지들이라면 사람이라도 그것들을 판별하는 것이 쉽지 않을 것이고 bayes error로 0%가 아니라 5% 혹은 13%로 설정하고 이 경우 훈련데이터에 대한 15%에러 나쁜 결과가 아닐 수 있다. 즉 설정한 bayes error에 따라 high bias이냐 low bias이냐의 판단이 달라진다.

앞서 high bias인 상황(직선)과 high variance(매우 복잡한 곡선)인 상황을 decision boundary를 그려 확인할 수 있었다. 그렇다면 위 세번째 상황과 같이 high bias & high variance인 상황의 decision boundary는 어떤 모습일까? 

.